# Setup

On google colab, you have to restart runtime after running the following line

In [1]:
!pip install omegaconf

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
##This is added because my Rdata uses Cdata for the covid data set.
##Rdata use Cdata function to load the data set, then convert it to regression problem
import os
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset')


In [4]:
# folder_name = os.getcwd()

In [5]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [6]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
torch.cuda.is_available()

True

# NCA and LMNN setup

In [9]:
pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.6 MB/s eta 0:00:00


In [10]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'


Newly added data sets for mental health (psychology):

Classification:
'psych_depression_physical_symptons',
'covid_anxious',
'covid_depressed'


In [11]:
dataset_name = 'covid_lonely_reg'
cfg = conf_file['dataset'][dataset_name]
#TODO need to add other covid data sets here.
if dataset_name in ['covid_depressed','covid_anxious','covid_physical','covid_lonely','covid_hopeless',
                    'psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)
   SOC1  SOC2A  SOC2B  SOC3A  SOC3B  SOC4A  SOC4B  PHYS8  PHYS1A  PHYS1B  \
0   3.0    4.0    4.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
1   3.0    3.0    3.0    1.0    3.0    2.0    2.0    2.0     2.0     2.0   
2   3.0    3.0    3.0    1.0    1.0    2.0    1.0    2.0     2.0     1.0   
3   3.0    2.0    2.0    1.0    1.0    2.0    1.0    3.0     2.0     2.0   
4   3.0    1.0    1.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
5   2.0    3.0    3.0    1.0    1.0    1.0    1.0    2.0     2.0     2.0   
6   2.0    4.0    3.0    1.0    1.0    2.0    2.0    NaN     1.0     2.0   
7   3

/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/reg_data.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ys = torch.tensor(ys, dtype=torch.float32)


In [12]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Rdata)
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

# Classification with NNKNN

In [55]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")


Unique values: [0 1 2 3]
Counts: [720 720 720 720]
Xs.size(): torch.Size([2880, 157])


In [56]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  model = Cmodel.NN_k_NN(X_train,
                         y_train,
                         cfg.ca_weight_sharing,
                         cfg.top_case_enabled,
                         cfg.top_k,
                         cfg.discount,
                         device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [57]:
def load_model_cls(X_train,y_train,cfg):
  # Define the model architecture
  model = Cmodel.NN_k_NN(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model

In [58]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = True

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)
    print("lmnn_acc: ",lmnn_acc)
    # continue

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)
  print("knn_acc: ", knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)
  print("nnknn acc: ", best_accuracy)
  break

print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


lmnn_acc:  0.3090277777777778
knn_acc:  0.28125
Epoch [2/1000], Loss: 1.3917
Epoch [4/1000], Loss: 1.3933
Epoch [6/1000], Loss: 1.3750
Epoch [8/1000], Loss: 1.3575
Epoch [10/1000], Loss: 1.4106
Epoch [12/1000], Loss: 1.3661
Epoch [14/1000], Loss: 1.2333
Epoch [16/1000], Loss: 1.4097
Epoch [18/1000], Loss: 1.1802
Epoch [20/1000], Loss: 1.4069
Epoch [22/1000], Loss: 1.0586
Epoch [24/1000], Loss: 1.0348
Epoch [26/1000], Loss: 1.0631
Epoch [28/1000], Loss: 1.2234
Epoch [30/1000], Loss: 0.9487
Epoch [32/1000], Loss: 1.2698
Epoch [34/1000], Loss: 1.2414
Epoch [36/1000], Loss: 1.1846
Epoch [38/1000], Loss: 1.0683
Epoch [40/1000], Loss: 1.2651
Epoch [42/1000], Loss: 1.2537
Epoch [44/1000], Loss: 0.9733
Epoch [46/1000], Loss: 1.3000
Epoch [48/1000], Loss: 1.1540
Epoch [50/1000], Loss: 1.1647
Epoch [52/1000], Loss: 0.9688
Epoch [54/1000], Loss: 1.2347
Epoch [56/1000], Loss: 1.2096
Epoch [58/1000], Loss: 1.0168
Epoch [60/1000], Loss: 0.9846
Epoch [62/1000], Loss: 1.1830
patience exceeded, loading

In [59]:
print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")

Average accuracy:0.358
KNN accuracy:0.281
LMNN/NCA accuracy:0.309


# Regression with NNKNN

In [13]:
unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")

Unique values: [-1.3414078  -0.44713593  0.44713593  1.3414078 ]
Counts: [720 720 720 720]
Xs.size(): torch.Size([2880, 157])


In [14]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, _, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)

  return best_accuracy, accuracy, top_k_average_accuracy, model

In [15]:
# prompt: load_model_reg()

def load_model_reg(X_train,y_train,cfg):
  # Define the model architecture
  model = Rmodel.NN_k_NN_regression(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      cfg.class_weight_sharing,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model


In [16]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)
  # break

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Epoch [2/1000], Loss: 0.7576
Epoch [4/1000], Loss: 0.6041
Epoch [6/1000], Loss: 1.0832
Epoch [8/1000], Loss: 0.7361
Epoch [10/1000], Loss: 0.9866
Epoch [12/1000], Loss: 0.8033
Epoch [14/1000], Loss: 0.3799
Epoch [16/1000], Loss: 0.6343
Epoch [18/1000], Loss: 0.4732
Epoch [20/1000], Loss: 0.7547
Epoch [22/1000], Loss: 0.6752
Epoch [24/1000], Loss: 0.3912
Epoch [26/1000], Loss: 0.3942
Epoch [28/1000], Loss: 0.3567
Epoch [30/1000], Loss: 0.8785
Epoch [32/1000], Loss: 0.4767
Epoch [34/1000], Loss: 0.4560
Epoch [36/1000], Loss: 0.5763
Epoch [38/1000], Loss: 0.3734
Epoch [40/1000], Loss: 0.5080
Epoch [42/1000], Loss: 0.2529
Epoch [44/1000], Loss: 0.3018
Epoch [46/1000], Loss: 0.8569
patience exceeded, loading best model
Epoch [2/1000], Loss: 1.1081
Epoch [4/1000], Loss: 0.7030
Epoch [6/1000], Loss: 0.4902
Epoch [8/1000], Loss: 0.6380
Epoch [10/1000], Loss: 0.6895
Epoch [12/1000], Loss: 0.8978
Epoch [14/1000], Loss: 0.2491
Epoch [16/1000], Loss: 0.4108
Epoch [18/1000], Loss: 0.7435
Epoch [20/

# Results Interpretation

In [17]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [18]:
print_model_features(model)

fa_layer.f1weight
tensor([4.3054, 3.3830, 2.9332, 3.0772, 3.2020, 0.0334, 0.8267, 3.5886, 4.4938,
        0.0374, 0.4696, 3.3637, 6.1253, 1.3574, 5.1637, 2.4555, 3.8334, 0.3158,
        2.9068, 2.5104, 0.0969, 0.2377, 3.6344, 3.8392, 2.3901, 0.5381, 2.1549,
        0.4553, 0.0356, 3.2979, 3.0341, 0.3186, 2.6029, 1.6466, 3.5997, 2.1260,
        5.3270, 2.0780, 4.2507, 3.0896, 3.6990, 0.0361, 3.2191, 5.8184, 4.6195,
        7.9962, 0.0181, 0.6586, 1.2627, 2.8021, 1.4553, 3.5561, 2.1857, 5.9375,
        0.0178, 0.0510, 1.0691, 3.3604, 5.6956, 0.0560, 0.0225, 2.5188, 2.2057,
        3.7461, 1.0906, 6.1584, 3.5896, 4.0770, 0.3392, 0.5209, 0.1669, 3.7024,
        0.1496, 5.6618, 0.0453, 1.2698, 0.0596, 3.7429, 2.8428, 1.0000, 2.4341,
        1.0000, 2.7945, 0.8153, 2.0559, 0.6911, 3.2749, 2.7245, 1.7029, 0.0254,
        0.7706, 2.5406, 0.7229, 1.9688, 4.2761, 0.1981, 0.0109, 0.0305, 0.0229,
        0.0373, 5.3246, 4.0876, 0.1112, 0.0245, 0.0151, 0.0219, 1.0000, 2.3410,
        0.4119, 0.9552

In [19]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
predicted_class

In [ ]:
y_test

In [ ]:
# prompt: accuracy comparing predicted_class and y_test

accuracy = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
print("Accuracy:", accuracy)


In [ ]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [ ]:
X_test[0]

In [ ]:
y_test[0]

In [ ]:
X_train[top_case_indices[0][0]]

In [ ]:
y_train[top_case_indices[0][0]]

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [ ]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

In [ ]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [ ]:
y_train[top_case_indices[0]]

In [ ]:
knn.predict(X_test)[0]

In [ ]:
indices = knn.kneighbors(X_test)[1][0]

In [ ]:
y_train[indices]

# Sanity Check


## Classification Neural Network

In [60]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 1024
num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network architecture for classification
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, num_classes)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x)


In [62]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  # Initialize the model, loss function, and optimizer
  model = NeuralNet(input_size, hidden_size, num_classes)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Training loop
  for epoch in range(epochs):
    epoch_msg = True
    training_total_acc = 0.0
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)

      # Backward and optimize
      _, predicted = torch.max(outputs, 1)
      training_total_acc += torch.sum(predicted == y_train_batch).item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_total_loss += loss.item()
      # if (i + 1) % 5 == 0
    if epoch == 0 or (epoch + 1) % 100 == 0:
      print(f"Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f} Acc: {training_total_acc/num_of_batches:.2f}")
    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      _, predicted = torch.max(outputs, 1)
      accuracy = torch.sum(predicted == y_test).item() / len(y_test)
      print(f'Accuracy on the test set: {accuracy * 100:.2f}%')
      if best_accuracy is None or accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epoch}], Test Loss: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best acc achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
  # break
print("Average accuracy:", np.mean(best_accuracies))

Epoch: 1, Training Loss: 1.64 Acc: 4.41
Accuracy on the test set: 31.94%
Accuracy on the test set: 32.99%
Accuracy on the test set: 35.42%
Accuracy on the test set: 32.64%
Accuracy on the test set: 39.58%
Accuracy on the test set: 37.85%
Accuracy on the test set: 34.03%
Accuracy on the test set: 36.81%
Accuracy on the test set: 39.24%
Accuracy on the test set: 31.94%
Accuracy on the test set: 37.15%
Accuracy on the test set: 34.72%
Accuracy on the test set: 37.50%
Accuracy on the test set: 37.15%
Accuracy on the test set: 32.64%
Accuracy on the test set: 36.11%
Accuracy on the test set: 40.62%
Accuracy on the test set: 40.62%
Accuracy on the test set: 35.76%
Accuracy on the test set: 35.76%
Accuracy on the test set: 41.67%
Accuracy on the test set: 36.11%
Accuracy on the test set: 42.01%
Accuracy on the test set: 36.81%
Accuracy on the test set: 39.24%
Accuracy on the test set: 32.99%
Accuracy on the test set: 42.01%
Accuracy on the test set: 40.28%
Accuracy on the test set: 32.29%
Acc

## Regression Neural Network

In [20]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 100
# num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [21]:
# prompt: a standard neural network with 3 fully connected layers for regression

import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self, input_size):
        super(RegressionNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, 1)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x).squeeze()

In [ ]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  model = RegressionNet(Xs.shape[1])
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  for epoch in range(epochs):
    epoch_msg = True
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)
      training_total_loss += loss.item()
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch == 0 or (epoch + 1) % 3 == 0:
      print(f'Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f}')

    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      if best_accuracy is None or loss.item() < best_accuracy:
        best_accuracy = loss.item()
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epochs}], Test Loss: {loss.item()}')
        # print(f'Loss on the test set: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best loss achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
print("Average accuracy:", np.mean(best_accuracies))


Epoch: 1, Training Loss: 1.16
Epoch: 3, Training Loss: 1.07
Epoch: 6, Training Loss: 1.01
Epoch: 9, Training Loss: 0.99
Epoch: 12, Training Loss: 0.97
Epoch: 15, Training Loss: 0.97
Epoch: 18, Training Loss: 0.96
Epoch: 21, Training Loss: 0.95
Epoch: 24, Training Loss: 0.95
Epoch: 27, Training Loss: 0.94
Epoch: 30, Training Loss: 0.94
Epoch: 33, Training Loss: 0.93
Epoch: 36, Training Loss: 0.93
Epoch: 39, Training Loss: 0.92
Epoch: 42, Training Loss: 0.92
Epoch: 45, Training Loss: 0.91
Epoch: 48, Training Loss: 0.91
Epoch: 51, Training Loss: 0.90
Epoch: 54, Training Loss: 0.90
Epoch: 57, Training Loss: 0.89
Epoch: 60, Training Loss: 0.89
Epoch: 63, Training Loss: 0.89
Epoch: 66, Training Loss: 0.88
Epoch: 69, Training Loss: 0.88
Epoch: 72, Training Loss: 0.87
Epoch: 75, Training Loss: 0.87
Epoch: 78, Training Loss: 0.86
Epoch: 81, Training Loss: 0.86
Epoch: 84, Training Loss: 0.86
Epoch: 87, Training Loss: 0.85
Epoch: 90, Training Loss: 0.85
Epoch: 93, Training Loss: 0.84
Epoch: 96, T